In [1]:
import pandas as pd
import os
import cv2
import shutil
import numpy as np

In [2]:
df1 = pd.read_excel('RUS_XLSX/batch_Aslan.xlsx', index_col=None)
df2 = pd.read_excel('RUS_XLSX/batch_Kundyz.xlsx', index_col=None)
df3 = pd.read_excel('RUS_XLSX/batch_Batyr.xlsx', index_col=None)

/home/user/ml_projects/Myrzakhan/KG_DOC_TRAINING/DOC_POLYGON_DETECTION/YOLOV5_POLYGON_DET/yolov5_pol_det_branch/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:211: UserWarning: Cell C3681 is marked as a date but the serial value 14467933.0 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/home/user/ml_projects/Myrzakhan/KG_DOC_TRAINING/DOC_POLYGON_DETECTION/YOLOV5_POLYGON_DET/yolov5_pol_det_branch/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:211: UserWarning: Cell C4090 is marked as a date but the serial value 6655481.0 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


In [3]:
df_List = [df1,df2,df3]
concat_df = pd.concat(df_List)

concat_df = concat_df.dropna()

concat_df = concat_df.reset_index(drop = True)

concat_df.shape

(12178, 11)

In [4]:
concat_df.head()

,filename,field_class,field_value,x1,y1,x2,y2,x3,y3,x4,y4
0,0a02980b-bc49-43ce-8e1b-b75f2dc7b6d9.jpg,place_of_issue,МВД ПО РЕСПУБЛИКЕ БАШКОРТОСТАН,173.0,231.0,606.0,199.0,610.0,237.0,173.0,262.0
1,0a02980b-bc49-43ce-8e1b-b75f2dc7b6d9.jpg,date_of_issue,0.09.2020,133.0,341.0,280.0,327.0,280.0,367.0,137.0,374.0
2,0a02980b-bc49-43ce-8e1b-b75f2dc7b6d9.jpg,authority,020-026,452.0,323.0,565.0,316.0,568.0,347.0,453.0,354.0
3,0a02980b-bc49-43ce-8e1b-b75f2dc7b6d9.jpg,iin,80 20 121051,636.0,231.0,668.0,226.0,676.0,450.0,634.0,452.0
4,0a02980b-bc49-43ce-8e1b-b75f2dc7b6d9.jpg,surname,ЧЕРНЫЙ,434.0,679.0,533.0,672.0,531.0,710.0,445.0,712.0


In [5]:
img_names_lst = concat_df.filename.unique().tolist()
print(len(img_names_lst))

1251


In [53]:
crops_names = list()
cnt = 0
i = 0
for item in img_names_lst:
    
    df_slice = concat_df.loc[concat_df['filename'] == item]

    if os.path.exists(os.path.join('RUS_FIELDS_TRAIN',item)):
        print(item)
        
        # print(item)
        img = cv2.imread(os.path.join('RUS_FIELDS_TRAIN',item))
        height = img.shape[0]
        width = img.shape[1]

        for index, row in df_slice.iterrows():
            if row['field_class']=='place_of_issue' or row['field_class']=='place_of_birth':
                if row['field_value'].count('$')>0:
                    splited = row['field_value'].split('$')
                    row_num = 0
                    divide_pix = int((min(row['y3'],row['y4']) - min(row['y2'],row['y1'])) / (row['field_value'].count('$')+1)) 
                    y1 = int(row['y1'])
                    y2 = int(row['y2'])
                    y3 = int(row['y2'])+ divide_pix
                    y4 = int(row['y1']) + divide_pix
                    for words in splited:
                        text = words.replace(' ','_')
                        text = '_'+text+'_'

                        crop_name = item.split('.')[0] + '_' +row['field_class']+ str(row_num) +'.' +  item.split('.')[1] + '&' + text
                        crops_names.append(crop_name)
                        crop_img_name = item.split('.')[0] + '_' +row['field_class']+ str(row_num) +'.' +  item.split('.')[1] 
                        row_num += 1

                        srcTri = np.array([[int(row['x4']),y4],
                                [int(row['x1']),y1],
                                [int(row['x2']),y2]]).astype(np.float32)
                
                        dstTri = np.array([[0,y3-y2], 
                                            [0,0], 
                                            [int(row['x2'])-int(row['x1']),0]
                                            ]).astype(np.float32)

                        warp_mat = cv2.getAffineTransform(srcTri, dstTri)
                        cropped = cv2.warpAffine(img, warp_mat, (int(row['x2'])-int(row['x1']), y3-y2))
                        i += 1
                        cv2.imwrite(os.path.join('rus_crops',crop_img_name), cropped)
                        
                        y1 += divide_pix
                        y2 += divide_pix
                        y3 += divide_pix
                        y4 += divide_pix

                else:
                    text = row['field_value'].replace(' ','_')
                    text = '_'+text+'_'
                        
                    crop_name = item.split('.')[0] + '_' +row['field_class']+ '.' +  item.split('.')[1] + '&' + text

                    crops_names.append(crop_name)
                    crop_img_name = item.split('.')[0] + '_' +row['field_class']+ '.' +  item.split('.')[1]

                    srcTri = np.array([[int(row['x4']),int(row['y4'])],
                                    [int(row['x1']),int(row['y1'])],
                                    [int(row['x2']),int(row['y2'])]]).astype(np.float32)
                    
                    dstTri = np.array([[0,int(row['y3'])-int(row['y2'])], 
                                        [0,0], 
                                        [int(row['x2'])-int(row['x1']),0]
                                        ]).astype(np.float32)

                    warp_mat = cv2.getAffineTransform(srcTri, dstTri)
                    cropped = cv2.warpAffine(img, warp_mat, (int(row['x2'])-int(row['x1']), int(row['y3'])-int(row['y2'])))
                    i += 1
                    cv2.imwrite(os.path.join('rus_crops',crop_img_name), cropped)



            else:
                if type(row['field_value']) == str:
                    text = row['field_value'].replace(' ','_')
                    text = '_'+text+'_'
                        
                    crop_name = item.split('.')[0] + '_' +row['field_class']+ '.' +  item.split('.')[1] + '&' + text

                    crops_names.append(crop_name)
                    crop_img_name = item.split('.')[0] + '_' +row['field_class']+ '.' +  item.split('.')[1]

                    srcTri = np.array([[int(row['x4']),int(row['y4'])],
                                    [int(row['x1']),int(row['y1'])],
                                    [int(row['x2']),int(row['y2'])]]).astype(np.float32)
                    
                    dstTri = np.array([[0,int(row['y3'])-int(row['y2'])], 
                                        [0,0], 
                                        [int(row['x2'])-int(row['x1']),0]
                                        ]).astype(np.float32)

                    warp_mat = cv2.getAffineTransform(srcTri, dstTri)
                    cropped = cv2.warpAffine(img, warp_mat, (int(row['x2'])-int(row['x1']), int(row['y3'])-int(row['y2'])))
                    i += 1
                    cv2.imwrite(os.path.join('rus_crops',crop_img_name), cropped)



0a02980b-bc49-43ce-8e1b-b75f2dc7b6d9.jpg
0a03766a-776b-4f4d-b21c-5eb77dfb9f08.jpg
0a04272a-c5ca-40fe-8aa1-836dae743ecb.jpg
0a056298-7144-47f4-9abc-38a659faa4b9.jpg
0a05705f-7b55-4a98-bd76-4ebd469931c5.jpg
0a065345-1b1c-4538-82db-9d4c806d778c.jpg
0a0655e4-d131-4921-be73-a9f45cf23129.jpg
0a06695a-6ae0-4bca-bd36-ea1aa43dbce9.jpg
0a0699b2-a1a4-4969-9407-46ceeb35707c.jpg
0a071727-cabd-4f6b-8732-c65a6da459c3.jpg
0a078154-fa37-4035-8146-1b6484186cd5.jpg
0a079583-389c-4ce7-82ab-926b7ffeb45c.jpg
0a0866ec-7b9d-450d-bdc2-3a51640fa6b9.jpg
0a093085-dc39-4923-b6c0-52153cd0d1f5.jpg
0a1008b0-2ec6-4bbd-b21e-84d7c6b55df6.jpg
0b0babc1-5726-443f-a0e4-33f1bf1da187.jpg
0b0bc9cc-fe7a-490c-951a-ffbf6adc5502.jpg
0b0cd24e-7ed1-4541-9a10-aaf2887e15e8.jpg
0b0d3f43-764b-419b-9fd5-c5176f46c02c.jpg
0b0cacb2-0cb9-49a4-8920-27a7bd792d96.jpg
0b09fbe1-00a8-44e5-8615-39f89f1ec44e.jpg
0b0a2eb9-498c-46dc-8acc-55c9a659ca95.jpg
0b0a317b-adf4-4959-a97e-80e8535f67fe.jpg
0b0a779b-c507-4f9e-8b0d-e5dc837e26cf.jpg
0b0b0e13-6d44-4e

In [54]:
print(len(crops_names))

14026


In [55]:
res = [*set(crops_names)]
print(len(res))

13895


In [56]:
new_res = list()
lst = list()
trash = list()
for item in res:
    nm = item.split('&')[0].split('.')[0]
    if nm not in new_res:
        new_res.append(nm)
        lst.append(item)
    elif nm in new_res:
        trash.append(item)

In [59]:
print(trash)

['0a561c35-443b-4050-a418-626049ed925b_patronymic.jpg&_АЛЕКСАНДРОВИЧ_', '0ad61dd0-7cba-4acb-9291-f6659aaa3ae4_date_of_issue.jpg&_06.03.1999_', '0c7b50aa-37d2-4fbe-ad93-d7e17aeafa44_place_of_issue1.jpg&__ОТДЕЛА_УФМС_РОССИИ_ПО_МУРМАНСКОЙ_ОБЛ_В__', '0abc6482-5494-45ba-b751-c79ff60bae11_date_of_issue.jpg&_380-006_', '0ab37fb8-cb1b-44de-a9e8-dab8c87cc67b_place_of_issue2.jpg&_В_КНЯЖПОГОСТСКОГО_РАЙОНА_', '0aa24be0-f3f0-4734-ab41-074a14b39154_place_of_issue0.jpg&_ОТДЕЛОМ_МИЛИЦИИ_ЧКАЛОВСКОГО_РУВД_', '0c3f270e-d8e8-4744-9dd7-6b29e50c39d1_authority.jpg&_60__08__391439_', '0b587525-f6fd-4f14-9114-b012e3abaafd_authority.jpg&_560-003_', '0ab8dcfc-e76b-4fc8-a51d-2c0a28ae8415_name.jpg&_МАРИЯ_', '0a9925f8-d231-4814-969b-dad5a19340f4_date_of_birth.jpg&_13.09.2007_', '0a648700-307d-4d68-9941-f22db28c91bb_place_of_birth0.jpg&_С._ОЛЬХОВ-ЛОГ_', '0b406203-da04-436a-a6e6-7ef8ee0a838d_date_of_birth.jpg&_28.11.1999_', '0ce9f99a-da6d-4249-b1d7-ed2184e619fb_place_of_birth1.jpg&__ШАКМИРСКИЙ_РАЙОН__', '0c3ceb10-3f9

In [58]:
with open('rus_crops.txt', 'w') as f:
    for line in res:
        f.write(f"{line}\n")

In [61]:
file = open('rus_crops_val.txt', 'r')

for line in file:
    name = line.split('&')[0]
    print(name)
    src_path = os.path.join('rus_crops_train', name)
    dst_path = os.path.join('rus_crops_val', name)
    shutil.move(src_path, dst_path)

0c43d675-3f80-4a05-ab53-211492c07d09_place_of_birth1.jpg
0ace3d42-dca0-488a-83d1-095f0e1306bc_place_of_birth1.jpg
0b416d49-5bd6-4b05-98b5-a2a15d1dab26_name.jpg
0cb5e5bf-0c4b-422b-b42c-784f21adc36e_place_of_birth1.jpg
0c7e24e7-8e76-4eba-9ff3-eab8f5801395_place_of_birth.jpg
0aacde4d-7da9-498a-a3f3-daefc5a4fa48_name.jpg
0ce222a4-5951-49fb-85b1-1e9b39e19bfe_place_of_birth0.jpg
0ce7d5ec-93aa-4fa4-ae55-06a6acc9f66d_surname.jpg
0abf3664-ce8a-45ea-94ca-e69c44934fca_place_of_issue1.jpg
0c3a005c-1db2-460e-8357-a8ec2b484437_name.jpg
0a66610e-9f6b-4b76-863c-81eec2539fd3_place_of_issue0.jpg
0cba2dfd-1a48-400c-95b8-e3a6a5bd70d4_place_of_issue0.jpg
0b55d158-221e-46d3-994e-e924720457de_place_of_birth.jpg
0a95484b-2762-4112-b654-b41c9aec588f_date_of_birth.jpg
0a9925f8-d231-4814-969b-dad5a19340f4_name.jpg
0a04272a-c5ca-40fe-8aa1-836dae743ecb_place_of_birth1.jpg
0ab6dd87-f5a9-4aca-a321-ff84bd496b2d_place_of_issue0.jpg
001f33fb-695d-41c5-b198-d0e3dfb686aa_patronymic.jpg
0cc93c0f-4cdc-4f19-959e-3f50fcfad9d